In [51]:
# Load the df from ckplus_dataset.csv

import pandas as pd
import numpy as np
import os

df = pd.read_csv('ckplus_dataset.csv')

In [53]:
# Take any random row from the df for testing
# In implementation, do this for all rows in the df

while True:
  sample_id = np.random.randint(0, len(df))
  sample = df.iloc[sample_id]
  emotion = sample['peak_emotion']
  
  if f'{emotion}.txt' in os.listdir():
    print(sample)
    break

line_features = []
angle_features = []
with open(f'{emotion}.txt', 'r') as file:
    for line in file:
        feature = line.strip().split()
        feature = [int(value) for value in feature[0][2:-1].split(',')]
        if len(feature) == 2:
            line_features.append(feature)
        else:
            angle_features.append(feature)

subject_id                        S022
session_id                           5
image_id                            31
peak_emotion                     anger
ckplus_labelled                      1
                          ...         
landmark_64        366.75939 293.84635
landmark_65         377.06463 295.2418
landmark_66         366.3825 294.79421
landmark_67        356.11855 297.15064
landmark_68        345.21262 295.74743
Name: 457, Length: 73, dtype: object


In [54]:
def euclidean_distance(a, b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def smaller_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    # return in degrees. remove the * 180 / np.pi to return in radians
    return np.arccos(cosine_angle) * 180 / np.pi

In [55]:
distances = []
angles = []

for feature in line_features:
    point1 = sample[f'landmark_{feature[0]}'].split(' ')
    point1 = [float(point1[0]), float(point1[1])]
    point2 = sample[f'landmark_{feature[1]}'].split(' ')
    point2 = [float(point2[0]), float(point2[1])]
    
    distance = euclidean_distance(point1, point2)
    # print(f'Distance between {feature[0]} and {feature[1]}: {distance}')
    distances.append(distance)
    
for feature in angle_features:
    point1 = sample[f'landmark_{feature[0]}'].split(' ')
    point1 = [float(point1[0]), float(point1[1])]
    point2 = sample[f'landmark_{feature[1]}'].split(' ')
    point2 = [float(point2[0]), float(point2[1])]
    point3 = sample[f'landmark_{feature[2]}'].split(' ')
    point3 = [float(point3[0]), float(point3[1])]
    
    angle = smaller_angle(point1, point2, point3)
    # print(f'Angle between {feature[0]}, {feature[1]} and {feature[2]}: {angle}')
    angles.append(angle)
    
# print(distances, angles)

In [56]:
# create new df with columns subject_id,session_id,image_id,peak_emotion,ckplus_labelled, distances, angles

data = []
data.append([sample['subject_id'], sample['session_id'], sample['image_id'], sample['peak_emotion'], sample['ckplus_labelled'], distances, angles])

new_df = pd.DataFrame(data, columns=['subject_id', 'session_id', 'image_id', 'peak_emotion', 'ckplus_labelled', 'distances', 'angles'])

print(new_df)

  subject_id  session_id  image_id peak_emotion  ckplus_labelled  \
0       S022           5        31        anger                1   

                                           distances  \
0  [12.77147026008358, 14.20876182255161, 33.8431...   

                                              angles  
0  [60.308644871495765, 59.13928037749884, 62.575...  
